In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import duckdb
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str(datetime.now().date() - timedelta(days=1))
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
stats = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

Today's date: 2025-11-24


In [2]:
%run ./common_utils.ipynb

In [4]:
def create_actuals_tbl(con):
    df_lines = pd.read_csv(f"../tables/{year}/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[(df_lines.Date == now)]
    df_log = pd.read_csv(f"../tables/{year}/season_gamelogs.csv")
    df_log['Date'] = pd.to_datetime(df_log.Date)
    df_log = df_log[(df_log.Date == now)]
    df_log = df_log.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
    df_log['PR'] = df_log.PTS + df_log.REB 
    df_log['PA'] = df_log.PTS + df_log.AST
    df_log['RA'] = df_log.REB + df_log.AST
    df_log['PRA'] = df_log.PTS + df_log.REB + df_log.AST
    df_log['STL_BLK'] = df_log.STL + df_log.BLK
    df = con.execute("""SELECT t2.*, t1.* EXCLUDE(Date, Team, Player, Pos) FROM df_log t1 JOIN df_lines t2 
                        ON t1.Team = t2.Team AND t1.Player = t2.Player""").fetchdf()
    cols = []
    for stat in stats:
        df[f'{stat}_Result'] = np.where(df[stat] > df[f'{stat}_line'], 'O', 'U')
        df[f'{stat}_Result'] = np.where(df[f'{stat}_line'].isnull(), np.nan, df[f'{stat}_Result'])
        df[f'{stat}_Diff'] = df[stat] - df[f'{stat}_line']
        cols.append(stat)
        cols.append(f'{stat}_line')
        cols.append(f'{stat}_Diff')
        cols.append(f'{stat}_Result')
    df = df[['game_id', 'Date', 'Team', 'Player', 'Pos', 'MP'] + cols]
    
    df_stats = pd.read_csv(f"../tables/{year}/parlay_stats.csv")
    df_stats['Date'] = pd.to_datetime(df_stats.Date).astype(str)
    lines = []
    diffs = []
    stats_cols = []
    
    for stat in stats:
        lines.append(f'{stat}_line')
        diffs.append(f'{stat}_Diff')
    for stat in stats:
        stats_cols.extend([
            f"Off_{stat}",
            f"Off_L5_{stat}",
            f"AVG_{stat}_H2H",
            f"Def_{stat}",
            f"Def_L5_{stat}",
            f"df_stats.{stat}_line",
            f"{stat}",
            f"df.{stat}_Diff",
            f"{stat}_Result"
        ])
    df_select = ", ".join(stats_cols)
    lines = str(lines).replace("[", "").replace("]", "")
    diffs = str(diffs).replace("[", "").replace("]", "")
    df_actuals = con.execute(f"""SELECT game_id, df.Date, df.Team, df.Player, df.Pos, MP, 
                                 {df_select} FROM df JOIN df_stats 
                                 ON df.Date = df_stats.Date AND df.Player = df_stats.Player
                                 """).fetchdf()
    partition_save_df(df_actuals, f"../tables/{year}/parlay_actuals.csv")
    display(df_actuals)

    return df_actuals
                      
df = create_actuals_tbl(con)

../tables/2025/parlay_actuals.csv saved!


,game_id,Date,Team,Player,Pos,MP,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,PTS,PTS_Diff,PTS_Result,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,AST,AST_Diff,AST_Result,Off_REB,Off_L5_REB,AVG_REB_H2H,Def_REB,Def_L5_REB,REB_line,REB,REB_Diff,REB_Result,Off_PR,Off_L5_PR,AVG_PR_H2H,Def_PR,Def_L5_PR,PR_line,PR,PR_Diff,PR_Result,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,PA,PA_Diff,PA_Result,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,RA,RA_Diff,RA_Result,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,PRA,PRA_Diff,PRA_Result,Off_TPM,Off_L5_TPM,AVG_TPM_H2H,Def_TPM,Def_L5_TPM,TPM_line,TPM,TPM_Diff,TPM_Result,Off_STL,Off_L5_STL,AVG_STL_H2H,Def_STL,Def_L5_STL,STL_line,STL,STL_Diff,STL_Result,Off_BLK,Off_L5_BLK,AVG_BLK_H2H,Def_BLK,Def_L5_BLK,BLK_line,BLK,BLK_Diff,BLK_Result,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,Def_L5_STL_BLK,STL_BLK_line,STL_BLK,STL_BLK_Diff,STL_BLK_Result
0,20251123_LAC_CLE,2025-11-23,LAC,James Harden,PG,31.70,28.400000,38.40,19.750000,17.846154,21.6,23.5,19,-4.5,U,8.533333,7.8,8.000000,6.769231,6.2,8.5,6,-2.5,U,6.428571,7.00,6.000000,4.000000,2.0,4.5,2,-2.5,U,34.400000,45.4,25.750000,21.846154,23.6,28.5,21,-7.5,U,36.933333,46.20,27.750000,24.615385,27.8,31.5,25,-6.5,U,14.533333,14.80,14.000000,10.769231,8.2,13.5,8,-5.5,U,42.933333,53.2,33.750000,28.615385,29.8,36.5,27,-9.5,U,4.133333,5.6,3.000000,1.846154,3.2,3.5,0,-3.5,U,1.500000,1.2,0.500000,1.076923,1.0,1.5,1,-0.5,U,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None
1,20251123_LAC_CLE,2025-11-23,CLE,Darius Garland,PG,29.47,14.750000,14.75,NaN,21.357143,25.0,17.5,8,-9.5,U,5.500000,5.5,NaN,5.357143,3.4,6.5,8,1.5,O,2.750000,2.75,NaN,4.071429,4.2,2.5,1,-1.5,U,17.500000,17.5,NaN,25.428571,29.2,20.5,9,-11.5,U,20.250000,20.25,NaN,26.714286,28.4,23.5,16,-7.5,U,8.250000,8.25,NaN,9.428571,7.6,8.5,9,0.5,O,23.000000,23.0,NaN,30.785714,32.6,26.5,17,-9.5,U,2.500000,2.5,NaN,3.000000,2.6,2.5,0,-2.5,U,1.500000,1.5,NaN,1.571429,1.6,0.5,0,-0.5,U,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None
2,20251123_SAS_PHO,2025-11-23,PHO,Collin Gillespie,PG,30.17,11.125000,13.60,5.600000,21.461538,27.2,13.5,15,1.5,O,5.062500,4.6,1.200000,4.615385,4.0,5.5,6,0.5,O,4.250000,5.60,1.800000,5.000000,4.8,4.5,3,-1.5,U,15.375000,19.2,7.400000,26.461538,32.0,18.5,18,-0.5,U,16.187500,18.20,6.800000,26.076923,31.2,19.5,21,1.5,O,9.312500,10.20,3.000000,9.615385,8.8,10.5,9,-1.5,U,20.437500,23.8,8.600000,31.076923,36.0,23.5,24,0.5,O,2.666667,2.4,1.200000,2.692308,4.6,2.5,3,0.5,O,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None
3,20251123_CHO_ATL,2025-11-23,ATL,Vit Krejci,PG,32.30,12.769231,12.40,6.714286,21.062500,25.0,10.5,1,-9.5,U,1.846154,2.2,1.285714,6.125000,6.2,2.5,4,1.5,O,2.230769,1.80,2.285714,4.125000,4.2,2.5,3,0.5,O,13.928571,14.0,9.000000,25.187500,29.2,13.5,4,-9.5,U,13.571429,14.00,8.000000,27.187500,31.2,12.5,5,-7.5,U,3.785714,3.20,3.571429,10.250000,10.4,5.5,7,1.5,O,15.642857,15.6,10.285714,31.312500,35.4,15.5,8,-7.5,U,3.307692,3.6,1.428571,2.875000,3.4,2.5,0,-2.5,U,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None
4,20251123_LAL_UTA,2025-11-23,LAL,Luka Doncic,PG,40.17,34.636364,31.80,26.777778,22.615385,26.8,32.5,33,0.5,O,9.000000,8.4,9.000000,7.538462,7.4,8.5,8,-0.5,U,8.545455,6.80,7.444444,3.846154,4.4,7.5,11,3.5,O,43.181818,38.6,34.222222,26.461538,31.2,40.5,44,3.5,O,43.636364,40.20,35.777778,30.153846,34.2,41.5,41,-0.5,U,17.545455,15.20,16.444444,11.384615,11.8,16.5,19,2.5,O,52.181818,47.0,43.222222,34.000000,38.6,49.5,52,2.5,O,3.454545,3.2,2.888889,3.307692,3.8,3.5,3,-0.5,U,2.333333,2.0,1.666667,1.692308,2.6,1.5,3,1.5,O,1.500000,1.5,0.333333,0.384615,0.6,0.5,0,-0.5,U,3.0,2.6,2.0,2.076923,3.2,2.5,3,0.5,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [21]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    print(f"==={stat}===")
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Player, Pos, MP, Opp, 
                            Off_{stat}, Off_L5_{stat}, AVG_{stat}_H2H, 
                            Def_{stat}, Def_L5_{stat}, {stat}_line, {stat}, 
                            {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL AND {stat}_Result = 'O'
                            ORDER BY {stat}_Diff DESC
                            LIMIT 15""").fetchdf())

===PTS===


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,PTS,PTS_Diff,PTS_Result
0,20251123_ORL_BOS,2025-11-23,ORL,1,Jett Howard,SF,29.08,BOS,4.555556,5.8,1.000000,17.428571,23.4,6.5,30,23.5,O
1,20251123_BRK_TOR,2025-11-23,BRK,0,Tyrese Martin,SG,24.85,TOR,8.692308,9.4,1.800000,15.000000,20.4,8.5,26,17.5,O
2,20251123_LAC_CLE,2025-11-23,LAC,1,Ivica Zubac,C,32.92,CLE,16.000000,17.8,NaN,15.928571,19.0,15.5,33,17.5,O
3,20251123_BRK_TOR,2025-11-23,BRK,0,Noah Clowney,PF,37.13,TOR,11.071429,14.8,8.000000,18.500000,14.8,12.5,22,9.5,O
4,20251123_ORL_BOS,2025-11-23,BOS,1,Anfernee Simons,SG,22.22,ORL,14.800000,16.4,16.250000,16.928571,15.0,13.5,23,9.5,O
5,20251123_CHO_ATL,2025-11-23,ATL,1,Dyson Daniels,SG,35.67,CHO,8.777778,9.4,9.666667,16.142857,10.8,12.5,22,9.5,O
6,20251123_LAC_CLE,2025-11-23,CLE,1,Donovan Mitchell,SG,32.65,LAC,30.333333,30.2,NaN,15.666667,16.0,27.5,37,9.5,O
7,20251123_CHO_ATL,2025-11-23,CHO,1,Kon Knueppel,SF,34.45,ATL,18.812500,23.6,NaN,20.470588,19.2,19.5,28,8.5,O
8,20251123_ORL_BOS,2025-11-23,BOS,1,Jaylen Brown,SF,37.43,ORL,27.437500,26.2,22.769231,19.235294,22.0,26.5,35,8.5,O
9,20251123_ORL_BOS,2025-11-23,BOS,1,Sam Hauser,PF,27.48,ORL,7.615385,4.4,4.769231,14.181818,10.0,6.5,14,7.5,O


===AST===


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,AST,AST_Diff,AST_Result
0,20251123_ORL_BOS,2025-11-23,BOS,1,Jaylen Brown,SF,37.43,ORL,4.062500,4.4,2.923077,3.235294,3.2,4.5,8,3.5,O
1,20251123_ORL_BOS,2025-11-23,BOS,1,Payton Pritchard,PG,33.53,ORL,4.937500,4.6,3.000000,7.466667,7.2,4.5,8,3.5,O
2,20251123_CHO_ATL,2025-11-23,ATL,1,Jalen Johnson,SF,39.57,CHO,7.187500,9.2,2.000000,3.363636,4.2,7.5,11,3.5,O
3,20251123_BRK_TOR,2025-11-23,TOR,0,Jamal Shead,PG,23.83,BRK,5.375000,6.2,6.800000,6.111111,5.4,5.5,9,3.5,O
4,20251123_ORL_BOS,2025-11-23,ORL,1,Tyus Jones,PG,24.68,BOS,2.000000,2.6,4.833333,6.416667,6.0,2.5,6,3.5,O
5,20251123_POR_OKC,2025-11-23,OKC,0,Chet Holmgren,PF,25.40,POR,1.727273,1.6,2.333333,3.461538,3.8,1.5,4,2.5,O
6,20251123_BRK_TOR,2025-11-23,BRK,0,Terance Mann,SG,20.75,TOR,4.153846,4.8,1.400000,3.823529,3.4,3.5,5,1.5,O
7,20251123_LAC_CLE,2025-11-23,CLE,1,Darius Garland,PG,29.47,LAC,5.500000,5.5,NaN,5.357143,3.4,6.5,8,1.5,O
8,20251123_CHO_ATL,2025-11-23,CHO,1,Sion James,SG,32.20,ATL,1.785714,2.0,NaN,3.538462,2.6,1.5,3,1.5,O
9,20251123_CHO_ATL,2025-11-23,ATL,1,Zaccharie Risacher,SF,21.60,CHO,2.500000,2.6,1.250000,3.363636,4.2,1.5,3,1.5,O


===REB===


...

,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,PA,PA_Diff,PA_Result
0,20251123_LAC_CLE,2025-11-23,LAC,1,Ivica Zubac,C,32.92,CLE,18.250000,20.0,NaN,18.857143,22.4,17.5,37,19.5,O
1,20251123_ORL_BOS,2025-11-23,BOS,1,Jaylen Brown,SF,37.43,ORL,31.500000,30.6,25.692308,22.470588,25.2,30.5,43,12.5,O
2,20251123_LAC_CLE,2025-11-23,CLE,1,Donovan Mitchell,SG,32.65,LAC,35.600000,35.2,NaN,20.266667,20.6,32.5,43,10.5,O
3,20251123_ORL_BOS,2025-11-23,BOS,1,Anfernee Simons,SG,22.22,ORL,16.500000,16.0,19.000000,20.571429,16.8,15.5,25,9.5,O
4,20251123_CHO_ATL,2025-11-23,ATL,1,Dyson Daniels,SG,35.67,CHO,14.333333,15.4,12.666667,20.642857,14.8,18.5,28,9.5,O
5,20251123_BRK_TOR,2025-11-23,TOR,0,Jamal Shead,PG,23.83,BRK,11.250000,11.0,13.600000,26.666667,23.8,11.5,20,8.5,O
6,20251123_CHO_ATL,2025-11-23,ATL,1,Jalen Johnson,SF,39.57,CHO,29.750000,34.2,12.857143,22.636364,25.6,31.5,39,7.5,O
7,20251123_LAL_UTA,2025-11-23,UTA,0,Keyonte George,PG,39.05,LAL,29.687500,32.2,23.250000,23.846154,28.0,27.5,35,7.5,O
8,20251123_CHO_ATL,2025-11-23,CHO,1,Kon Knueppel,SF,34.45,ATL,21.687500,26.6,NaN,23.470588,23.0,23.5,31,7.5,O
9,20251123_POR_OKC,2025-11-23,OKC,0,Shai Gilgeous-Alexander,PG,30.00,POR,38.437500,37.0,36.875000,28.909091,27.2,38.5,44,5.5,O


===RA===


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,RA,RA_Diff,RA_Result
0,20251123_LAC_CLE,2025-11-23,LAC,1,Ivica Zubac,C,32.92,CLE,13.437500,15.0,NaN,12.500000,11.6,13.5,22,8.5,O
1,20251123_ORL_BOS,2025-11-23,BOS,1,Payton Pritchard,PG,33.53,ORL,9.187500,9.6,5.615385,12.266667,11.4,8.5,13,4.5,O
2,20251123_LAL_UTA,2025-11-23,LAL,0,Austin Reaves,SG,39.97,UTA,12.666667,10.2,9.571429,9.600000,8.2,9.5,14,4.5,O
3,20251123_POR_OKC,2025-11-23,OKC,0,Chet Holmgren,PF,25.40,POR,9.416667,7.2,8.166667,9.692308,10.6,9.5,13,3.5,O
4,20251123_LAL_UTA,2025-11-23,UTA,0,Keyonte George,PG,39.05,LAL,11.125000,11.4,9.625000,9.769231,10.0,9.5,13,3.5,O
5,20251123_POR_OKC,2025-11-23,OKC,0,Isaiah Hartenstein,C,22.22,POR,14.250000,13.2,15.400000,12.833333,12.4,13.5,17,3.5,O
6,20251123_CHO_ATL,2025-11-23,CHO,1,Sion James,SG,32.20,ATL,4.562500,5.6,NaN,7.538462,5.2,5.5,9,3.5,O
7,20251123_CHO_ATL,2025-11-23,ATL,1,Dyson Daniels,SG,35.67,CHO,11.388889,12.4,6.333333,8.500000,7.2,11.5,15,3.5,O
8,20251123_LAC_CLE,2025-11-23,CLE,1,Donovan Mitchell,SG,32.65,LAC,10.066667,10.6,NaN,9.000000,10.4,10.5,14,3.5,O
9,20251123_ORL_BOS,2025-11-23,ORL,1,Franz Wagner,SF,27.67,BOS,10.294118,10.6,11.272727,9.214286,9.8,10.5,13,2.5,O


===PRA===


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,PRA,PRA_Diff,PRA_Result
0,20251123_LAC_CLE,2025-11-23,LAC,1,Ivica Zubac,C,32.92,CLE,29.437500,32.8,NaN,28.428571,30.6,29.5,55,25.5,O
1,20251123_LAC_CLE,2025-11-23,CLE,1,Donovan Mitchell,SG,32.65,LAC,40.400000,40.8,NaN,24.666667,26.4,37.5,51,13.5,O
2,20251123_CHO_ATL,2025-11-23,ATL,1,Dyson Daniels,SG,35.67,CHO,20.166667,21.8,16.000000,24.642857,18.0,24.5,37,12.5,O
3,20251123_ORL_BOS,2025-11-23,BOS,1,Anfernee Simons,SG,22.22,ORL,18.750000,19.0,21.000000,24.214286,19.2,18.5,30,11.5,O
4,20251123_ORL_BOS,2025-11-23,BOS,1,Jaylen Brown,SF,37.43,ORL,37.187500,38.0,31.692308,28.705882,32.0,36.5,47,10.5,O
5,20251123_LAL_UTA,2025-11-23,UTA,0,Keyonte George,PG,39.05,LAL,33.687500,36.4,26.875000,27.000000,31.8,30.5,40,9.5,O
6,20251123_CHO_ATL,2025-11-23,CHO,1,Kon Knueppel,SF,34.45,ATL,27.500000,31.2,NaN,29.352941,29.8,29.5,37,7.5,O
7,20251123_ORL_BOS,2025-11-23,BOS,1,Payton Pritchard,PG,33.53,ORL,25.687500,28.4,15.769231,35.933333,40.4,25.5,32,6.5,O
8,20251123_CHO_ATL,2025-11-23,ATL,1,Nickeil Alexander-Walker,SG,38.57,CHO,25.500000,30.2,12.750000,24.642857,18.0,26.5,32,5.5,O
9,20251123_POR_OKC,2025-11-23,OKC,0,Shai Gilgeous-Alexander,PG,30.00,POR,43.312500,41.2,42.250000,33.818182,31.4,43.5,49,5.5,O


In [22]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    print(f"==={stat}===")
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Player, Pos, MP, Opp, 
                            Off_{stat}, Off_L5_{stat}, AVG_{stat}_H2H, 
                            Def_{stat}, Def_L5_{stat}, {stat}_line, {stat}, 
                            {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL AND {stat}_Result = 'U'
                            ORDER BY {stat}_Diff
                            LIMIT 15""").fetchdf())

===PTS===


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,PTS,PTS_Diff,PTS_Result
0,20251123_LAL_UTA,2025-11-23,LAL,0,Deandre Ayton,C,13.43,UTA,16.500000,16.00,11.250000,17.461538,22.4,14.5,2,-12.5,U
1,20251123_POR_OKC,2025-11-23,POR,0,Deni Avdija,SF,23.77,OKC,25.937500,25.60,13.571429,19.285714,17.4,23.5,11,-12.5,U
2,20251123_POR_OKC,2025-11-23,OKC,0,Chet Holmgren,PF,25.40,POR,18.583333,18.80,11.666667,18.769231,23.4,17.5,5,-12.5,U
3,20251123_ORL_BOS,2025-11-23,ORL,1,Goga Bitadze,C,0.00,BOS,6.562500,6.40,3.583333,17.545455,13.4,9.5,0,-9.5,U
4,20251123_ORL_BOS,2025-11-23,ORL,1,Franz Wagner,SF,27.67,BOS,23.470588,25.80,21.909091,17.428571,23.4,24.5,15,-9.5,U
5,20251123_LAC_CLE,2025-11-23,CLE,1,Darius Garland,PG,29.47,LAC,14.750000,14.75,NaN,21.357143,25.0,17.5,8,-9.5,U
6,20251123_POR_OKC,2025-11-23,POR,0,Toumani Camara,PF,26.57,OKC,12.500000,13.40,9.555556,18.333333,15.2,13.5,4,-9.5,U
7,20251123_CHO_ATL,2025-11-23,ATL,1,Vit Krejci,PG,32.30,CHO,12.769231,12.40,6.714286,21.062500,25.0,10.5,1,-9.5,U
8,20251123_CHO_ATL,2025-11-23,ATL,1,Zaccharie Risacher,SF,21.60,CHO,11.733333,9.40,12.000000,19.272727,21.4,12.5,5,-7.5,U
9,20251123_ORL_BOS,2025-11-23,BOS,1,Luka Garza,C,0.00,ORL,8.230769,9.60,4.400000,14.000000,16.0,7.5,0,-7.5,U


===AST===


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,AST,AST_Diff,AST_Result
0,20251123_POR_OKC,2025-11-23,POR,0,Deni Avdija,SF,23.77,OKC,5.687500,8.0,3.714286,3.428571,3.4,6.5,1,-5.5,U
1,20251123_SAS_PHO,2025-11-23,SAS,0,De'Aaron Fox,PG,33.38,PHO,6.571429,7.6,7.125000,6.642857,7.4,7.5,3,-4.5,U
2,20251123_SAS_PHO,2025-11-23,PHO,0,Royce O'Neale,SF,33.70,SAS,2.875000,2.2,1.666667,3.600000,5.0,3.5,1,-2.5,U
3,20251123_LAL_UTA,2025-11-23,UTA,0,Isaiah Collier,PG,22.70,LAL,6.500000,6.2,6.200000,6.615385,6.2,5.5,3,-2.5,U
4,20251123_ORL_BOS,2025-11-23,ORL,1,Goga Bitadze,C,0.00,BOS,2.083333,2.2,1.083333,3.454545,3.6,2.5,0,-2.5,U
5,20251123_LAC_CLE,2025-11-23,CLE,1,Jaylon Tyson,SG,22.77,LAC,1.916667,2.4,NaN,4.600000,4.6,2.5,0,-2.5,U
6,20251123_LAC_CLE,2025-11-23,LAC,1,James Harden,PG,31.70,CLE,8.533333,7.8,8.000000,6.769231,6.2,8.5,6,-2.5,U
7,20251123_BRK_TOR,2025-11-23,TOR,0,Immanuel Quickley,PG,30.35,BRK,6.125000,6.0,5.666667,6.111111,5.4,6.5,4,-2.5,U
8,20251123_BRK_TOR,2025-11-23,TOR,0,RJ Barrett,SF,20.15,BRK,3.937500,3.8,3.833333,3.823529,4.0,3.5,1,-2.5,U
9,20251123_POR_OKC,2025-11-23,OKC,0,Cason Wallace,SG,26.77,POR,2.571429,2.4,1.666667,3.800000,4.0,2.5,1,-1.5,U


===REB===


...

,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,PA,PA_Diff,PA_Result
0,20251123_POR_OKC,2025-11-23,POR,0,Deni Avdija,SF,23.77,OKC,31.625000,33.60,17.285714,22.714286,20.8,30.5,12,-18.5,U
1,20251123_ORL_BOS,2025-11-23,ORL,1,Goga Bitadze,C,0.00,BOS,7.647059,8.40,4.666667,21.000000,17.0,11.5,0,-11.5,U
2,20251123_POR_OKC,2025-11-23,OKC,0,Chet Holmgren,PF,25.40,POR,20.166667,20.00,14.000000,22.230769,27.2,19.5,9,-10.5,U
3,20251123_ORL_BOS,2025-11-23,ORL,1,Franz Wagner,SF,27.67,BOS,27.411765,30.20,27.000000,20.428571,26.6,29.5,20,-9.5,U
4,20251123_POR_OKC,2025-11-23,POR,0,Toumani Camara,PF,26.57,OKC,15.250000,15.20,11.111111,21.866667,18.6,15.5,7,-8.5,U
5,20251123_CHO_ATL,2025-11-23,ATL,1,Vit Krejci,PG,32.30,CHO,13.571429,14.00,8.000000,27.187500,31.2,12.5,5,-7.5,U
6,20251123_LAL_UTA,2025-11-23,UTA,0,Lauri Markkanen,PF,35.75,LAL,32.625000,37.60,24.500000,24.176471,30.8,29.5,22,-7.5,U
7,20251123_LAC_CLE,2025-11-23,CLE,1,Darius Garland,PG,29.47,LAC,20.250000,20.25,NaN,26.714286,28.4,23.5,16,-7.5,U
8,20251123_BRK_TOR,2025-11-23,TOR,0,Brandon Ingram,SF,35.75,BRK,25.125000,24.60,24.000000,22.823529,28.6,24.5,18,-6.5,U
9,20251123_CHO_ATL,2025-11-23,ATL,1,Zaccharie Risacher,SF,21.60,CHO,13.400000,12.00,13.250000,22.636364,25.6,14.5,8,-6.5,U


===RA===


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,RA,RA_Diff,RA_Result
0,20251123_ORL_BOS,2025-11-23,ORL,1,Goga Bitadze,C,0.00,BOS,6.529412,7.4,5.416667,14.000000,13.8,10.5,0,-10.5,U
1,20251123_ORL_BOS,2025-11-23,BOS,1,Neemias Queta,C,5.85,ORL,10.125000,11.6,3.083333,11.733333,14.0,10.5,2,-8.5,U
2,20251123_POR_OKC,2025-11-23,POR,0,Deni Avdija,SF,23.77,OKC,12.375000,15.6,8.142857,10.000000,9.0,13.5,6,-7.5,U
3,20251123_BRK_TOR,2025-11-23,TOR,0,RJ Barrett,SF,20.15,BRK,8.937500,9.6,8.166667,8.764706,9.8,9.5,3,-6.5,U
4,20251123_LAC_CLE,2025-11-23,LAC,1,James Harden,PG,31.70,CLE,14.533333,14.8,14.000000,10.769231,8.2,13.5,8,-5.5,U
5,20251123_LAL_UTA,2025-11-23,UTA,0,Isaiah Collier,PG,22.70,LAL,9.750000,9.6,9.200000,9.769231,10.0,8.5,4,-4.5,U
6,20251123_CHO_ATL,2025-11-23,CHO,1,Collin Sexton,SG,36.07,ATL,7.000000,5.8,8.000000,7.538462,5.2,7.5,4,-3.5,U
7,20251123_LAC_CLE,2025-11-23,CLE,1,Jaylon Tyson,SG,22.77,LAC,6.250000,9.0,NaN,9.000000,10.4,7.5,4,-3.5,U
8,20251123_SAS_PHO,2025-11-23,SAS,0,De'Aaron Fox,PG,33.38,PHO,10.000000,11.0,13.625000,11.142857,10.4,11.5,8,-3.5,U
9,20251123_POR_OKC,2025-11-23,OKC,0,Cason Wallace,SG,26.77,POR,5.000000,3.4,4.777778,7.066667,6.6,5.5,3,-2.5,U


===PRA===


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,PRA,PRA_Diff,PRA_Result
0,20251123_POR_OKC,2025-11-23,POR,0,Deni Avdija,SF,23.77,OKC,38.312500,41.2,21.714286,29.285714,26.4,37.5,17,-20.5,U
1,20251123_ORL_BOS,2025-11-23,ORL,1,Goga Bitadze,C,0.00,BOS,12.705882,13.8,9.000000,31.545455,27.2,19.5,0,-19.5,U
2,20251123_ORL_BOS,2025-11-23,BOS,1,Neemias Queta,C,5.85,ORL,19.625000,23.0,6.500000,25.733333,30.0,20.5,8,-12.5,U
3,20251123_LAC_CLE,2025-11-23,LAC,1,John Collins,PF,29.88,CLE,17.062500,15.6,NaN,36.750000,32.6,19.5,8,-11.5,U
4,20251123_BRK_TOR,2025-11-23,TOR,0,RJ Barrett,SF,20.15,BRK,28.500000,28.2,19.333333,27.764706,34.4,29.5,19,-10.5,U
5,20251123_LAC_CLE,2025-11-23,LAC,1,James Harden,PG,31.70,CLE,42.933333,53.2,33.750000,28.615385,29.8,36.5,27,-9.5,U
6,20251123_LAC_CLE,2025-11-23,CLE,1,Darius Garland,PG,29.47,LAC,23.000000,23.0,NaN,30.785714,32.6,26.5,17,-9.5,U
7,20251123_POR_OKC,2025-11-23,OKC,0,Chet Holmgren,PF,25.40,POR,28.000000,26.0,19.833333,28.461538,34.0,27.5,18,-9.5,U
8,20251123_ORL_BOS,2025-11-23,ORL,1,Franz Wagner,SF,27.67,BOS,33.764706,36.4,33.181818,26.642857,33.2,36.5,28,-8.5,U
9,20251123_CHO_ATL,2025-11-23,ATL,1,Zaccharie Risacher,SF,21.60,CHO,16.066667,14.8,17.000000,27.545455,32.2,17.5,10,-7.5,U
